<a href="https://colab.research.google.com/github/quinn-dougherty/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/module1-logistic-regression/Copy_of_LS_DS1_231_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Lecture - Where Linear goes Wrong
### Return of the Titanic 🚢

You've likely already explored the rich dataset that is the Titanic - let's use regression and try to predict survival with it. The data is [available from Kaggle](https://www.kaggle.com/c/titanic/data), so we'll also play a bit with [the Kaggle API](https://github.com/Kaggle/kaggle-api).

In [0]:
!pip install kaggle

In [13]:
# Note - you'll also have to sign up for Kaggle and authorize the API
# https://github.com/Kaggle/kaggle-api#api-credentials

# This essentially means uploading a kaggle.json file
# For Colab we can have it in Google Drive
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

# You also have to join the Titanic competition to have access to the data
!kaggle competitions download -c titanic

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 22.6MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 25.5MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.15MB/s]


ls: cannot access 'content/drive/My Drive/': No such file or directory


In [15]:
# How would we try to do this with linear regression?
import pandas as pd

train_df = pd.read_csv('train.csv').dropna()
test_df = pd.read_csv('test.csv').dropna()  # Unlabeled, for Kaggle submission

train_df.head()

PassengerId  Survived  Pclass  \
1             2         1       1   
3             4         1       1   
6             7         0       1   
10           11         1       3   
11           12         1       1   

                                                 Name     Sex   Age  SibSp  \
1   Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
3        Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
6                             McCarthy, Mr. Timothy J    male  54.0      0   
10                    Sandstrom, Miss. Marguerite Rut  female   4.0      1   
11                           Bonnell, Miss. Elizabeth  female  58.0      0   

    Parch    Ticket     Fare Cabin Embarked  
1       0  PC 17599  71.2833   C85        C  
3       0    113803  53.1000  C123        S  
6       0     17463  51.8625   E46        S  
10      1   PP 9549  16.7000    G6        S  
11      0    113783  26.5500  C103        S

In [16]:
train_df.describe()

PassengerId    Survived      Pclass         Age       SibSp  \
count   183.000000  183.000000  183.000000  183.000000  183.000000   
mean    455.366120    0.672131    1.191257   35.674426    0.464481   
std     247.052476    0.470725    0.515187   15.643866    0.644159   
min       2.000000    0.000000    1.000000    0.920000    0.000000   
25%     263.500000    0.000000    1.000000   24.000000    0.000000   
50%     457.000000    1.000000    1.000000   36.000000    0.000000   
75%     676.000000    1.000000    1.000000   47.500000    1.000000   
max     890.000000    1.000000    3.000000   80.000000    3.000000   

            Parch        Fare  
count  183.000000  183.000000  
mean     0.475410   78.682469  
std      0.754617   76.347843  
min      0.000000    0.000000  
25%      0.000000   29.700000  
50%      0.000000   57.000000  
75%      1.000000   90.000000  
max      4.000000  512.329200

In [17]:
from sklearn.linear_model import LinearRegression

X = train_df[['Pclass', 'Age', 'Fare']]
y = train_df.Survived

linear_reg = LinearRegression().fit(X, y)
linear_reg.score(X, y)

0.08389810726550917

In [18]:
linear_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

array([0.79543117, 0.58610823, 0.67595121, 0.793829  , 0.62090522,
       0.72542107, 0.59848968, 0.58734245, 0.48567063, 0.77627736,
       0.84211887, 0.57052439, 0.7754689 , 0.96621114, 0.70287941,
       0.57673837, 0.72321391, 0.75894755, 0.77968041, 0.50246003,
       0.49858077, 0.7474959 , 0.3542282 , 0.61648435, 0.71300224,
       0.66294608, 0.53175333, 0.77397395, 0.68419387, 0.68395536,
       0.52041202, 0.56814038, 0.79586606, 0.81372012, 0.61068545,
       0.57260627, 0.52525981, 0.58055388, 0.45584728, 0.67976208,
       0.8226707 , 0.84286197, 0.96189157, 0.66724612, 0.68589478,
       0.61846513, 0.63455044, 0.68275686, 0.65738372, 0.45198998,
       0.59988596, 0.63845908, 0.63132487, 0.7888473 , 0.60126246,
       0.79714045, 0.78713803, 0.54643775, 0.42823635, 0.7711724 ,
       0.53552976, 0.55608044, 0.54480459, 0.57031915, 0.65080369,
       0.77958926, 0.6371013 , 0.70993488, 0.71493598, 0.60375943,
       0.54407206, 0.48186138, 0.76576089, 0.75456305, 0.53968

In [19]:
linear_reg.coef_

array([-0.08596295, -0.00829314,  0.00048775])

In [20]:
import numpy as np

test_case = np.array([[1, 5, 500]])  # Rich 5-year old in first class
linear_reg.predict(test_case)

array([1.14845883])

In [21]:
np.dot(linear_reg.coef_, test_case.reshape(-1, 1)) + linear_reg.intercept_

array([1.14845883])

In [22]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7103825136612022

In [23]:
log_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [24]:
log_reg.predict(test_case)[0]

1

In [25]:
help(log_reg.predict)

Help on method predict in module sklearn.linear_model.base:

predict(X) method of sklearn.linear_model.logistic.LogisticRegression instance
    Predict class labels for samples in X.
    
    Parameters
    ----------
    X : array_like or sparse matrix, shape (n_samples, n_features)
        Samples.
    
    Returns
    -------
    C : array, shape [n_samples]
        Predicted class label per sample.



In [26]:
log_reg.predict_proba(test_case)[0]

array([0.02485552, 0.97514448])

In [27]:
# What's the math?
log_reg.coef_

array([[-0.0455017 , -0.02912513,  0.0048037 ]])

In [28]:
log_reg.intercept_

array([1.45878264])

In [0]:
# The logistic sigmoid "squishing" function, implemented to accept numpy arrays
def sigmoid(x):
  return 1 / (1 + np.e**(-x))

In [30]:
sigmoid(log_reg.intercept_ + np.dot(log_reg.coef_, np.transpose(test_case)))

array([[0.97514448]])

So, clearly a more appropriate model in this situation! For more on the math, [see this Wikipedia example](https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study).

For live - let's tackle [another classification dataset on absenteeism](http://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work) - it has 21 classes, but remember, scikit-learn LogisticRegression automatically handles more than two classes. How? By essentially treating each label as different (1) from some base class (0).

In [31]:
# Live - let's try absenteeism!
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00445/Absenteeism_at_work_AAA.zip
!unzip Absenteeism_at_work_AAA.zip	

--2019-01-22 21:44:30--  http://archive.ics.uci.edu/ml/machine-learning-databases/00445/Absenteeism_at_work_AAA.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66136 (65K) [application/zip]
Saving to: ‘Absenteeism_at_work_AAA.zip’

Absenteeism_at_work 100%[===================>]  64.59K   258KB/s    in 0.3s    

2019-01-22 21:44:30 (258 KB/s) - ‘Absenteeism_at_work_AAA.zip’ saved [66136/66136]

Archive:  Absenteeism_at_work_AAA.zip
  inflating: Absenteeism_at_work.arff  
  inflating: Absenteeism_at_work.csv  
  inflating: Absenteeism_at_work.xls  
  inflating: Attribute Information.docx  
  inflating: UCI_ABS_TEXT.docx       


In [32]:
!head Absenteeism_at_work.csv

ID;Reason for absence;Month of absence;Day of the week;Seasons;Transportation expense;Distance from Residence to Work;Service time;Age;Work load Average/day ;Hit target;Disciplinary failure;Education;Son;Social drinker;Social smoker;Pet;Weight;Height;Body mass index;Absenteeism time in hours
11;26;7;3;1;289;36;13;33;239.554;97;0;1;2;1;0;1;90;172;30;4
36;0;7;3;1;118;13;18;50;239.554;97;1;1;1;1;0;0;98;178;31;0
3;23;7;4;1;179;51;18;38;239.554;97;0;1;0;1;0;0;89;170;31;2
7;7;7;5;1;279;5;14;39;239.554;97;0;1;2;1;1;0;68;168;24;4
11;23;7;5;1;289;36;13;33;239.554;97;0;1;2;1;0;1;90;172;30;2
3;23;7;6;1;179;51;18;38;239.554;97;0;1;0;1;0;0;89;170;31;2
10;22;7;6;1;361;52;3;28;239.554;97;0;1;1;1;0;4;80;172;27;8
20;23;7;6;1;260;50;11;36;239.554;97;0;1;4;1;0;0;65;168;23;4
14;19;7;2;1;155;12;14;34;239.554;97;0;1;2;1;0;0;95;196;25;40


In [33]:
absent_df = pd.read_table('Absenteeism_at_work.csv', sep=';')
absent_df.head()

ID  Reason for absence  Month of absence  Day of the week  Seasons  \
0  11                  26                 7                3        1   
1  36                   0                 7                3        1   
2   3                  23                 7                4        1   
3   7                   7                 7                5        1   
4  11                  23                 7                5        1   

   Transportation expense  Distance from Residence to Work  Service time  Age  \
0                     289                               36            13   33   
1                     118                               13            18   50   
2                     179                               51            18   38   
3                     279                                5            14   39   
4                     289                               36            13   33   

   Work load Average/day             ...              Disciplinary failure  \
0                 239.554            ...                                 0   
1                 239.554            ...                                 1   
2                 239.554            ...                                 0   
3                 239.554            ...                                 0   
4                 239.554            ...                                 0   

   Education  Son  Social drinker  Social smoker  Pet  Weight  Height  \
0          1    2               1              0    1      90     172   
1          1    1               1              0    0      98     178   
2          1    0               1              0    0      89     170   
3          1    2               1              1    0      68     168   
4          1    2               1              0    1      90     172   

   Body mass index  Absenteeism time in hours  
0               30                          4  
1               31                          0  
2               31                          2  
3               24                          4  
4               30                          2  

[5 rows x 21 columns]

In [34]:
absent_df.shape

(740, 21)

In [35]:
LogisticRegression

sklearn.linear_model.logistic.LogisticRegression

In [36]:
absent_df.columns

Index(['ID', 'Reason for absence', 'Month of absence', 'Day of the week',
       'Seasons', 'Transportation expense', 'Distance from Residence to Work',
       'Service time', 'Age', 'Work load Average/day ', 'Hit target',
       'Disciplinary failure', 'Education', 'Son', 'Social drinker',
       'Social smoker', 'Pet', 'Weight', 'Height', 'Body mass index',
       'Absenteeism time in hours'],
      dtype='object')

In [37]:
X = absent_df.drop('Reason for absence', axis='columns')
y = absent_df['Reason for absence']

absent_log1 = LogisticRegression().fit(X, y)
absent_log1.score(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5013513513513513

In [38]:
absent_log1

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [39]:
absent_log1.coef_

array([[-1.27508146e-02, -2.20793886e-01, -4.08404457e-02,
         4.22616389e-01,  7.64613632e-03, -6.18658809e-03,
        -3.11523480e-02,  5.52309565e-02, -1.01707258e-02,
        -1.14781041e-01,  2.51984519e+00,  1.41303285e-02,
         3.56702636e-01,  2.27778616e-01,  1.80478520e-01,
         1.33441356e-01, -9.82098540e-03,  5.23702470e-02,
         7.24744784e-02, -3.15221513e+00],
       [ 5.11128074e-02,  1.77166203e-01, -5.01201750e-01,
        -9.34413067e-02,  8.70070989e-03,  2.90683453e-02,
        -8.19496921e-02,  1.30910624e-01,  3.69193210e-03,
         3.58478924e-02, -7.77198534e-01,  9.21419989e-01,
         2.74046853e-01, -6.53744541e-01, -2.74959514e-01,
        -4.46143318e-01,  1.88638790e-01, -7.60409657e-02,
        -6.85430554e-01,  4.55734002e-03],
       [ 1.74477093e-01,  5.91768593e-01,  4.46355493e-02,
         4.29016838e-01,  5.97802716e-02, -2.02281583e-01,
        -4.44906133e-01, -2.65239556e-01,  8.55486140e-03,
        -4.07755000e-01, -6.7

In [0]:
?LogisticRegression

## Assignment - real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

In [2]:
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

--2019-01-23 14:22:56--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.13, 2001:620:5ca1:2ff::ce53
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip’

fma_metadata.zip    100%[===================>] 341.81M  16.4MB/s    in 24s     

2019-01-23 14:23:20 (14.4 MB/s) - ‘fma_metadata.zip’ saved [358412441/358412441]

Archive:  fma_metadata.zip
 bunzipping: fma_metadata/README.txt  
 bunzipping: fma_metadata/checksums  
 bunzipping: fma_metadata/not_found.pickle  
 bunzipping: fma_metadata/raw_genres.csv  
 bunzipping: fma_metadata/raw_albums.csv  
 bunzipping: fma_metadata/raw_artists.csv  
 bunzipping: fma_metadata/raw_tracks.csv  
 bunzipping: fma_metadata/tracks.csv  
 bunzipping: fma_metadata/genres.csv  
 bunzipping: fma_metadata/raw_echonest.cs

In [3]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import datetime as dt


tracks = pd.read_csv('fma_metadata/tracks.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
tracks.describe()

Unnamed: 0   album              album.1              album.2  \
count       106575  106575               103046                70295   
unique      106575      29                14341                 3670   
top         143640       0  2015-01-26 13:04:57  2008-01-01 00:00:00   
freq             1   71187                  310                  667   

               album.3  album.4  album.5  \
count            15296   106575   106575   
unique             623       65    14929   
top     Ernie Indradat        0       -1   
freq               876    45753      805   

                                                  album.6  album.7  \
count                                               83150   106575   
unique                                              11076    11351   
top     <p class="p1" style="margin: 0px; padding: 8px...       -1   
freq                                                  310     3130   

           album.8    ...     \
count        18061    ...      
unique         761    ...      
top     Joe Belock    ...      
freq           855    ...      

                                                 track.10 track.11  track.12  \
count                                                2350   106575     15025   
unique                                               1587    18977        45   
top     <p><a href="http://www.myspace.com/theshambler...      320        en   
freq                                                   22       67     14255   

                                                 track.13 track.14  \
count                                              106488   106575   
unique                                                114    15340   
top     Attribution-Noncommercial-Share Alike 3.0 Unit...       97   
freq                                                19250      110   

             track.15 track.16              track.17  track.18  track.19  
count             312   106575                  1264    106575    106574  
unique             67      331                   136      2452     94987  
top     Apache Tomcat        1  Victrola Dog (ASCAP)        []  Untitled  
freq               44    10459                   465     83078       298  

[4 rows x 53 columns]

In [5]:
pd.set_option('display.max_columns', 55)  # enough columns
tracks.head()

Unnamed: 0     album              album.1              album.2   album.3  \
0        NaN  comments         date_created        date_released  engineer   
1   track_id       NaN                  NaN                  NaN       NaN   
2          2         0  2008-11-26 01:44:45  2009-01-05 00:00:00       NaN   
3          3         0  2008-11-26 01:44:45  2009-01-05 00:00:00       NaN   
4          5         0  2008-11-26 01:44:45  2009-01-05 00:00:00       NaN   

     album.4 album.5      album.6  album.7   album.8 album.9  \
0  favorites      id  information  listens  producer    tags   
1        NaN     NaN          NaN      NaN       NaN     NaN   
2          4       1      <p></p>     6073       NaN      []   
3          4       1      <p></p>     6073       NaN      []   
4          4       1      <p></p>     6073       NaN      []   

               album.10 album.11 album.12               artist  \
0                 title   tracks     type    active_year_begin   
1                   NaN      NaN      NaN                  NaN   
2  AWOL - A Way Of Life        7    Album  2006-01-01 00:00:00   
3  AWOL - A Way Of Life        7    Album  2006-01-01 00:00:00   
4  AWOL - A Way Of Life        7    Album  2006-01-01 00:00:00   

          artist.1           artist.2  \
0  active_year_end  associated_labels   
1              NaN                NaN   
2              NaN                NaN   
3              NaN                NaN   
4              NaN                NaN   

                                            artist.3  artist.4  \
0                                                bio  comments   
1                                                NaN       NaN   
2  <p>A Way Of Life, A Collective of Hip-Hop from...         0   
3  <p>A Way Of Life, A Collective of Hip-Hop from...         0   
4  <p>A Way Of Life, A Collective of Hip-Hop from...         0   

              artist.5   artist.6 artist.7    artist.8    artist.9  \
0         date_created  favorites       id    latitude    location   
1                  NaN        NaN      NaN         NaN         NaN   
2  2008-11-26 01:42:32          9        1  40.0583238  New Jersey   
3  2008-11-26 01:42:32          9        1  40.0583238  New Jersey   
4  2008-11-26 01:42:32          9        1  40.0583238  New Jersey   

     artist.10                                          artist.11 artist.12  \
0    longitude                                            members      name   
1          NaN                                                NaN       NaN   
2  -74.4056612  Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...      AWOL   
3  -74.4056612  Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...      AWOL   
4  -74.4056612  Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...      AWOL   

                                           artist.13 artist.14  \
0                                   related_projects      tags   
1                                                NaN       NaN   
2  The list of past projects is 2 long but every1...  ['awol']   
3  The list of past projects is 2 long but every1...  ['awol']   
4  The list of past projects is 2 long but every1...  ['awol']   

                                 artist.15       artist.16       set   set.1  \
0                                  website  wikipedia_page     split  subset   
1                                      NaN             NaN       NaN     NaN   
2  http://www.AzillionRecords.blogspot.com             NaN  training   small   
3  http://www.AzillionRecords.blogspot.com             NaN  training  medium   
4  http://www.AzillionRecords.blogspot.com             NaN  training   small   

      track   track.1   track.2              track.3              track.4  \
0  bit_rate  comments  composer         date_created        date_recorded   
1       NaN       NaN       NaN                  NaN                  NaN   
2    256000         0       NaN  2008-11-26 01:48:12  2008-11-26 00:00:00   
3    256000         0    

In [6]:
tracks.shape

(106576, 53)

This is the biggest data you've played with so far, and while it does generally fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

### Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.

In [10]:
real_feats_list = [tracks[tracks.columns[k]].iloc[0] for k in range(len(tracks.columns))]

# for k in range(len(tracks.columns)): 
#   print(k, tracks[tracks.columns[k]].iloc[0])

tracks.isna().sum()

real_feats_list

tracks.columns

def up_to_prd(s): 
  if len(s)==0: return ''
  elif s[0]=='.': return ''
  else: return s[0] + up_to_prd(s[1:])

prefices = ['track_id'] + [up_to_prd(feat) for feat in tracks.columns[1:]]

ind_pairs = list(zip(prefices, real_feats_list))

index = pd.MultiIndex.from_tuples(ind_pairs, names=['first', 'second'])

df = (tracks
      .drop([0,1], axis=0)
      .T
      .set_index(index)
      .T)

df.head()

#for k in range(2,df.shape[0]): 
#  assert '<' in df.album.information.iloc[k]

#list(range(2,df.shape[0]))

#df['<' not in df.album.information.values].head()

KeyError: ignored

In [76]:
# drop column that invovles html tags 


MultiIndex(levels=[['album', 'artist', 'set', 'track', 'track_id'], ['active_year_begin', 'active_year_end', 'associated_labels', 'bio', 'bit_rate', 'comments', 'composer', 'date_created', 'date_recorded', 'date_released', 'duration', 'engineer', 'favorites', 'genre_top', 'genres', 'genres_all', 'id', 'information', 'interest', 'language_code', 'latitude', 'license', 'listens', 'location', 'longitude', 'lyricist', 'members', 'name', 'number', 'producer', 'publisher', 'related_projects', 'split', 'subset', 'tags', 'title', 'tracks', 'type', 'website', 'wikipedia_page']],
           labels=[[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [-1, 5, 7, 9, 11, 12, 16, 17, 22, 29, 34, 35, 36, 37, 0, 1, 2, 3, 5, 7, 12, 16, 20, 23, 24, 26, 27, 31, 34, 38, 39, 32, 33, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 17, 18, 19, 21, 22, 25, 28, 30, 34, 35]],
           names=['first', 'second'])

In [0]:
onehot_genres = pd.get_dummies(df['track']['genre_top'])

oh_g_tuples = list(zip(['genre_top']*onehot_genres.shape[1], onehot_genres.columns))

onehot_genres_multidx = onehot_genres.T.set_index(pd.MultiIndex.from_tuples(oh_g_tuples)).T
#onehot_genres_multidx.head()

#pd.concat([df, onehot_genres], axis=1).head()

onehot_genres.dtypes

oh_g_tuples

onehot_genres_multidx.shape

df_wgenres = pd.concat([df.drop('genre_top', axis=1, level='second'), onehot_genres_multidx], axis=1)


In [47]:
df_wgenres.genre_top.head()


df_wgenres.dtypes

#df_wgenres.album.date_created.apply(lambda x: x.to_datetime())

#pd.to_datetime(df_wgenres.album[['date_created', 'date_released']])


def up_to_spc(s): 
  s = str(s)
  if len(s)==0: return ''
  elif s[0]==' ': return ''
  else: return s[0] + up_to_spc(s[1:])

import datetime as dt
def todatetoord(feat): 
  df_wgenres.album[feat] = (pd.to_datetime(df_wgenres.album[feat]
                                           .apply(up_to_spc))
                            .apply(dt.datetime.toordinal))
  pass
  
#df_wgenres.album.date_created = pd.to_datetime(df_wgenres.album.date_created.apply(up_to_spc))
#df_wgenres.album.date_released = pd.to_datetime(df_wgenres.album.date_released)


todatetoord('date_created')
todatetoord('date_released')

df_wgenres.dtypes


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


first      second             
track_id   NaN                    object
album      comments               object
           date_created           object
           date_released          object
           engineer               object
           favorites              object
           id                     object
           information            object
           listens                object
           producer               object
           tags                   object
           title                  object
           tracks                 object
           type                   object
artist     active_year_begin      object
           active_year_end        object
           associated_labels      object
           bio                    object
           comments               object
           date_created           object
           favorites              object
           id                     object
           latitude               object
           location       

In [25]:
lgr = LogisticRegression() 

X = df_wgenres

y = df_wgenres.genre_top['Hip-Hop']


y.shape, X.shape


log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: ignored

In [12]:
''

Object `in` not found.
